# Functional Programming

In Python, everything you interact with is an object. From functions to immutable data types, they are all objects. So, when I say functional programming, I mean from the perspective of not creating custom objects through classes that we define ourself. That will be covered in the next chapter.

## Operators

The first thing I want to talk about are the operators used to perform math and logic.

### Math Operators


### Bitwise Operators

### Logical Operators 

### Self Application Shorthand

All symbol based operators have a shorthand form to apply the action to the current value. For example: x = x + 1 can also be written as x += 1. Below you can see many of them in action.

In [8]:
# I know the below prints aren't PEP 8, but it is cleaner for this case
x = 1024                                      ;print("x = {}".format(x))
x += 1                                        ;print("x = {}".format(x))
x -= 10                                       ;print("x = {}".format(x))
x *= 2                                        ;print("x = {}".format(x))
x **= 3                                       ;print("x = {}".format(x))
x /= 9                                        ;print("x = {}".format(x))
x //= 2                                       ;print("x = {}".format(x))
x = int(x)  # Reset back to an integer
x &= 0b11101                                  ;print("x = {}".format(x))
x |= 0b00010                                  ;print("x = {}".format(x))
x ^= 0b10101                                  ;print("x = {}".format(x))
x >>= 2                                       ;print("x = {}".format(x))
x <<= 2                                       ;print("x = {}".format(x))

x = 1024
x = 1025
x = 1015
x = 2030
x = 8365427000
x = 929491888.8888888
x = 464745944.0
x = 24
x = 26
x = 15
x = 3
x = 12


## Conditionals and Looping

### If Statement

### For Loop

### While Loop

## Functions

### Standard Functions

### Generators